In [113]:
from model_synthdata_inversion import *
from itertools import product
from myiapws import iapws1992, iapws1995
from scipy.integrate import solve_ivp
from scipy.optimize import fmin, minimize
from joblib import Parallel, delayed

import numpy as np
import matplotlib.pyplot as plt
import time
import warnings

cm = 1 / 2.54

## True conditions

In [123]:
rho0 = 0.5   # careful, density will vary with temperature!
Cp0  = 1885  # careful, Cp0 will vary with temperatur!e
Rp0  = 462   # careful, Rp0 will vary with temperatur!e
Pa0  = 86000 # Atmospheric pressure at altitude of la Soufrière
theta0 = np.pi/2
n0   = 0.05  # Fumarole plumes are always 95% vapour?
Ta0  = 291   # Tair 2-year average at Sanner
npts = 501
s    = np.linspace(0, 250, npts)
Tt   = iapws1995.Tt

# variables
N  = 21
R0 = np.linspace(0.1, 1, N)  
T0 = np.linspace(80 + Tt, 160 + Tt, N)
u0 = np.linspace(0.1, 100, N)

rho0true = .5
R0true = .5
T0true = Tt + 96
u0true = 10
Q0true = rho0true * u0true * R0true**2 * np.pi
M0true = Q0true * u0true
E0true = Q0true * Cp0 * T0true
V0true = [Q0true, M0true, E0true, theta0, Pa0, n0]

sol_true = solve_ivp(derivs, [s[0], s[-1]], V0true, t_eval=s)

# Produce "true" data values
sol  = sol_true
rho  = density_fume(sol.t, sol.y)
T    = temperature_fume(sol.t, sol.y) - Tt
Cp   = heat_capacity(sol.t, sol.y)
b    = sol.y[0] / np.sqrt(rho * sol.y[1])
u    = sol.y[1] / sol.y[0]
theta = sol.y[3]

cutoff = b.argmax()
s = s[:cutoff]
b = b[:cutoff]
u = u[:cutoff]
T = T[:cutoff]
theta = theta[:cutoff]

sigtheta, sigb, sigT = np.pi / 20, .5, .5  # rad, m and K
Cd_inv = np.diag(np.array([1 / sigtheta**2 * np.ones_like(theta),
                           1 / sigb**2 * np.ones_like(b),
                           1 / sigT**2 * np.ones_like(T)]).ravel())

solp = np.array([theta, b, T])
noise = np.random.randn(*solp.shape)  # Gaussian noise, _N_(0,1)
sol_noise = (solp.T + noise.T * (sigtheta, sigb, sigT)).T
d    = sol_noise.flatten()  # array of data
Gm   = produce_Gm(sol, s, cutoff)
Gm_d = Gm - d

fig0, ax0 = plt.subplots(figsize=(10*cm, 10*cm))
ax0.plot(solp.T, s, '-')
ax0.set_xlabel('Plume parameters')
ax0.set_ylabel(r'Distance along plume axis, $s$')
ax0.legend((r'$\theta$', r'$b$', r'$T$'))

plt.gca().set_prop_cycle(None)  # reset colour cycle
ax0.plot(sol_noise.T, s, '.')

fig0.tight_layout()

E = objective_fn(Gm, d, np.diag(Cd_inv), mode='abs')
print(E)

ValueError: could not broadcast input array from shape (3,501) into shape (3,489)

In [121]:
# produce_Gm(sol, s, cutoff)
print(len(sol.t), len(s), cutoff)


501 489 489


In [112]:
def parallel_job(u0, R0, T0):
    warnings.filterwarnings('ignore')
    V0   = [1, 1, Cp0 * T0, 1, 1, .05]  # required for routines
    rho0 = density_fume(0, V0)
    Q0   = rho0 * np.pi * R0**2 * u0
    M0   = Q0 * u0
    E0   = Q0 * Cp0 * T0
    V0   = [Q0, M0, E0, np.pi/2, Pa0, .05]

    sol  = solve_ivp(derivs, [s[0], s[-1]], V0, t_eval=s)
    
    Gm   = produce_Gm(sol, cutoff)

    return objective_fn(Gm, d, Cd_inv, mode='lstsq'), V0


njobs = 16
ngrid = 5
u0 = np.linspace(.1, 100, ngrid)
R0 = np.linspace(.1, 10,  ngrid)
T0 = np.linspace(60, 150, ngrid) + Tt

sequence = [u0, R0, T0]

t = time.time()
sequence = [u0, R0, T0]
objFn, initialConds = [], []

# results = Parallel(n_jobs=njobs)(delayed(parallel_job)(u0, R0, T0) 
#                                  for (u0,R0,T0) in list(product(*sequence)))
for (u0_, R0_, T0_) in list(product(*sequence)):
    print(u0_, R0_, T0_)
    results = parallel_job(u0_, R0_, T0_)
    objFn.append(results[0])
    initialConds.append(results[1])

print("Job ran in %.3f s" % (time.time() - t))

# objFn, initialConds = [], []

# for result in results:
#     objFn.append(result[0])
#     initialConds.append(result[1])

# initialConds = np.array(initialConds)
# objFn = np.array(objFn).reshape((-1, nGrid))

Reloading 'model_synthdata_inversion'.
0.1 0.1 333.16


TypeError: 'NoneType' object cannot be interpreted as an integer

In [110]:
print(u0_, R0_, T0_)
V0   = [1, 1, Cp0 * T0_, 1, 1, .05]  # required for routines
rho0 = density_fume(0, V0)
Q0   = rho0 * np.pi * R0_**2 * u0_

M0   = Q0 * u0_
E0   = Q0 * Cp0 * T0_
V0   = [Q0, M0, E0, np.pi/2, Pa0, .05]
print(V0)

sol = solve_ivp(derivs, [s[0], s[-1]], V0, t_eval=s)
len(sol.t)

Gm = produce_Gm(sol, s)
solp = Gm.reshape(3, -1)
plt.plot(solp.T, sol.t, '-')
plt.legend((r'$\theta$', r'$b$', r'$T$'))

0.1 0.1 333.16
[2.041056579024855e-08, 2.041056579024855e-09, 0.012817970026010305, 1.5707963267948966, 86000, 0.05]


TypeError: 'NoneType' object cannot be interpreted as an integer

In [104]:
Gm = produce_Gm(sol)
# solp[Gm.reshape(3, -1).shape] = solp.reshape(3, -1)

# solp[-1:len(s), :] = -9999



(3, 489)